# Report for data wrangling

7 february 2023\
Ali Benchemsi\
Gerard van Konijnenburg

# Libraries and options

In [1]:
import numpy as np
import pandas as pd
from sodapy import Socrata
from configparser import ConfigParser
pd.options.display.max_rows = 20
np.set_printoptions(precision = 4, suppress = True)

# Data loading & wrangling

Idee Gerard zaterdag 21-1

We hebben een heleboel data van taxiritten per rit, maar weerdata per dag voor ruim 100 weerstations. Ik stel voor om taxiritten te aggregeren tot 1 record per dag, zone van vertrek en zone van aankomst. Dit kunnen we dan tegen weerdata aanhouden en kijken of er een relatie in zit.

De weerdata heeft veel gegevens over het weer, dus daar moeten we ook wat zinnigs over vinden. Ik ben al met de taxiritten aan het stoeien, wil jij je eerst richten op de weerdata?

## Yellow cab data

In [2]:
#Read app token from config file
parser = ConfigParser()
parser.read('configuration.txt')
app_token = parser.get('ny_opendata', 'app_token')

In [3]:
#Load taxirides data
print('Loading data for taxi rides')
start = pd.Timestamp.now()
      
#Dataset constants
client = Socrata("data.cityofnewyork.us", app_token)
client.timeout = 30 #increase timeout value to 30 seconds
dataset = "m6nq-qud6" #2021 Yellow Taxi Trip Data
year_dataset = 2021

#Get departure zones
print('Loading departure zones', end = '\r')
query = '''
    select
        distinct pulocationid as zone_from
    order by
        zone_from
    '''
result = client.get(dataset, query = query)
zones = pd.DataFrame(result).squeeze()
num_zones = len(zones.index)
limit = str(num_zones * 366)

#Function that retrieves the data for one departure zone
def get_zonedata(zone_from):
    query = '''
    select
        date_trunc_ymd(tpep_pickup_datetime) as pickup_date,
        count(zone_from) as trips,
        avg(passenger_count) as avg_passengers,
        avg(trip_distance) as avg_distance,
        avg(trip_distance * passenger_count) / avg_passengers as avg_passengerdistance,
        pulocationid as zone_from,
        dolocationid as zone_to
    where
        zone_from = ''' + str(zone_from) + '''
        and passenger_count IS NOT NULL
        and passenger_count > 0
        and trip_distance > 0
    group by
        pickup_date,
        zone_from,
        zone_to
    limit ''' + limit
    
    result = client.get(dataset, query = query)
    return(pd.DataFrame(result))

#initialize dataframe with data
print('Loading data for departure zone 1/', str(num_zones), end = "\r")
df_taxirides = get_zonedata(zones[0])

#add data for other zones
i = 1
          
while i < (num_zones - 1):
    i += 1
    print('Loading data for departure zone', str(i+1), '/', str(num_zones), end = "\r")
    df_zone = get_zonedata(zones[i])
    df_taxirides = df_taxirides.append(df_zone, ignore_index = True)

end = pd.Timestamp.now()
duration = end - start
print(' '*80)
print('Taxidata loaded in ' + str(duration.seconds) + ' seconds')

Loading data for taxi rides
                                                                                
Taxidata loaded in 205 seconds


In [4]:
#write result to csv
df_taxirides.to_csv('data\taxirides.csv', index = False)

In [5]:
#Taxirides data wrangling

#Te implementeren
#colommen naar juiste datatype, alles staat in string

## Taxi zones

In [6]:
#Load taxizone data

#Te implementeren
#bron: https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc
#download data in een geschikt formaat voor data wrangling hieronder
#uitzoeken of er iets is met longitude, latitude data er al in
#data lezen

In [7]:
#Taxizone data wrangling

#Te implementeren
#bereken latitude and longitude voor elke zone zodat we in een volgende stap afstand tot de weerstations kunnen bepalen

## Weather data

In [8]:
#Load weather data

#Te implementeren
#lees weerdata (zie in overleaf map 'data')
#implementatie geeft nu een waarschuwing voor mixed datatupe in colom 29 en 55, uitzoeken hoe dit te ondervangen

#Implemenatie
df_weather = pd.read_csv('data\weatherdata.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (29,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
#Weather data wrangling

#Te implementeren
#geen idee, nog niet naar gekeken